### setup

In [ ]:
from __future__ import division

import os
import urllib, cStringIO

import pymongo as pm
import numpy as np
import scipy.stats as stats
import pandas as pd
import json
import re

from PIL import Image
import base64
import sys

from IPython.display import clear_output

## plotting
import matplotlib
from matplotlib import pylab, mlab, pyplot
%matplotlib inline
from IPython.core.pylabtools import figsize, getfigs
plt = pyplot
import seaborn as sns
sns.set_context('talk')
sns.set_style('white')

## svg rendering 
import ast
from svgpathtools import parse_path, wsvg, svg2paths



#### paths etc.


In [ ]:
# directory & file hierarchy
proj_dir = os.path.abspath('../')
stimulus_dir = os.getcwd()
analysis_dir = os.path.join(proj_dir,'analysis')
results_dir = os.path.join(proj_dir,'results')
plot_dir = os.path.join(results_dir,'plots')
csv_dir = os.path.join(results_dir,'csv')
exp_dir = os.path.abspath(os.path.join(proj_dir,'experiments'))
sketch_dir = os.path.abspath(os.path.join(proj_dir,'sketches'))

## add helpers to python path
if os.path.join(proj_dir,'analysis','python') not in sys.path:
    sys.path.append(os.path.join(proj_dir,'analysis','python'))
    
if not os.path.exists(results_dir):
    os.makedirs(results_dir)
    
if not os.path.exists(plot_dir):
    os.makedirs(plot_dir)   
    
if not os.path.exists(csv_dir):
    os.makedirs(csv_dir)       
    
# Assign variables within imported analysis helpers
import df_generation_helpers as h
if sys.version_info[0]>=3:
    from importlib import reload
reload(h)

#### load in group data csv

In [ ]:
path_to_group_data = os.path.join(results_dir,'graphical_conventions.csv')
x = pd.read_csv(path_to_group_data)

### render sketches using svg data

In [ ]:
import svg_rendering_helpers as srh

In [ ]:
## extract sketch identifying info
gseries = x['gameID'].map(str)
nseries = x['trialNum'].map(str).apply(lambda x: x.zfill(2))
rseries = x['repetition'].map(str).apply(lambda x: x.zfill(2))
tseries = x['target'].map(str)

## build list of image filenames
fname_list = ['{}_{}_{}'.format(i,j,k) for (i,j,k) in zip(gseries,rseries,tseries)]

## convert svg string strings into svg string list
svg_string_list = [ast.literal_eval(i) for i in x.svgString.values]

In [ ]:
## for debugging purposes, define a shortened version of the svg string list and fname list
short_svg = svg_string_list[:5]
short_fnames = fname_list[:5]

In [ ]:
## render out svg
reload(srh)
reallyRun = 0
if reallyRun:
    for this_fname,this_svg in zip(fname_list,svg_string_list):    
        srh.render_svg(this_svg,base_dir=sketch_dir,out_fname= '{}.svg'.format(this_fname))    
        clear_output(wait=True)

In [ ]:
## get svg path list for rendered out svg
svg_paths = srh.generate_svg_path_list(os.path.join(sketch_dir,'svg'))

In [ ]:
## convert all svg to png
reload(srh)
srh.svg_to_png(svg_paths,base_dir=sketch_dir)

### upload stims to s3

In [ ]:
import boto
bucket_name = 'graphical-conventions-sketches'
path_to_png = os.path.join(sketch_dir,'png')
runThis = 1
if runThis:
    conn = boto.connect_s3()
    b = conn.create_bucket(bucket_name) ### if bucket already exists, then get_bucket, else create_bucket
    for ind,im in enumerate(os.listdir(path_to_png)):
        if im[-3:]=='png':
            print ind, im
            k = b.new_key(im)
            k.set_contents_from_filename(os.path.join(path_to_png,im))
            k.set_acl('public-read')
            clear_output(wait=True)

### build stimulus dictionary

### upload to mongo

In [ ]:
dataset_name = 'graphical_conventions_sketches'
### next todo is to upload this JSON to initialize the new stimulus collection
print('next todo is to upload this JSON to initialize the new stimulus collection...')
import json
J = json.loads(open('{}.js'.format(dataset_name),mode='ru').read())

In [ ]:
print 'dataset_name: {}'.format(dataset_name)
print len(J)

In [ ]:
# set vars 
auth = pd.read_csv('auth.txt', header = None) # this auth.txt file contains the password for the sketchloop user
pswd = auth.values[0][0]
user = 'sketchloop'
host = 'rxdhawkins.me' ## cocolab ip address

# have to fix this to be able to analyze from local
conn = pm.MongoClient('mongodb://sketchloop:' + pswd + '@127.0.0.1')
db = conn['stimuli']
coll = db[dataset_name]

In [ ]:
## actually add data now to the database
reallyRun = 0
if reallyRun:
    for (i,j) in enumerate(J):
        if i%100==0:
            print ('%d of %d' % (i,len(J)))
        coll.insert_one(j)